In [53]:
from utils.pyre import db, storage
from openpyxl import load_workbook
import pandas as pd 
import numpy as np

In [54]:
def get_data(file,sheet_name,first,last):
    wb = load_workbook(filename=file, 
                   read_only=True,
                   data_only=True)
    ws = wb[sheet_name]

    # Read the cell values into a list of lists
    data_rows = []
    for row in ws[first:last]:
        data_cols = []
        for cell in row:
            data_cols.append(cell.value)
        data_rows.append(data_cols)

    # # Transform into dataframe
    # import pandas as pd
    db_df = pd.DataFrame(data_rows)
    # return db_df

In [55]:
import random
import qrcode
import string

def add_qr(name, scan_id):
    input_data = scan_id
    #Creating an instance of qrcode
    qr = qrcode.QRCode(
            version=1,
            box_size=10,
            border=5)
    qr.add_data(input_data)
    qr.make(fit=True)
    img = qr.make_image(fill='black', back_color='white')
    filename = f'assets/qr_codes/{name}_qr.png'
    img.save(filename)
    storage.child(f"{name}/{name}_qr").put(filename)
    qr_url = storage.child(f"{name}/{name}_qr").get_url(None)
    # print(qr_url)
    return qr_url

def generate():
        S = 8  # number of characters in the string.  
        # call random.choices() string module to find the string in Uppercase + numeric data.  
        ran = 'ac&@%!'+''.join(random.choices(string.ascii_uppercase + string.digits, k = S))    
        x= str(ran)
        return x


In [56]:
wb = load_workbook(filename='weyo.xlsx', 
                read_only=True,
                data_only=True)
ws = wb['Sheet1']

# Read the cell values into a list of lists
data_rows = []
for row in ws['A2':'K5']:
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    data_rows.append(data_cols)

# # Transform into dataframe
# import pandas as pd
db_df = pd.DataFrame(data_rows)

In [13]:
dba_df = get_data('doto.xlsx','Sheet1','A2','K5')


In [57]:
db_df

,0,1,2,3,4,5,6,7,8,9,10
0,H Lazaroi (H 293),0.53,0.254,0.51,0.37,0.495,0.181,0.77,0.26,0.154,0.63
1,H lobiii,1.04,0.400,0.62,0.43,0.330,0.350,0.10,0.21,0.150,0.13
2,Hoya 310 (Davao 1),0.23,0.130,0.10,0.05,0.030,0.050,0.04,0.09,0.080,0.04
3,Hoya 198 wms,0.39,0.150,0.15,0.08,0.050,0.070,0.03,0.08,0.060,0.04


In [24]:
for i in range(len(db_df)):
    print(db_df[0][i])




H.Lazaroi (H.293)
H.lobiii
Hoya 310 (Davao 1)
Hoya 198 wms


In [46]:
for i in range(len(db_df)):
    # name = db_df[0][i]
    name = db_df[0][i]

    print(name)
    data = {
        "Passport Data": {
            'Name': f'{name}'
        }
    }
    db.child('Hoya').child(f'{name}').set(data)
    

H.Lazaroi (H.293)


HTTPError: [Errno 400 Client Error: Bad Request for url: https://pnri-demeter-default-rtdb.firebaseio.com/Hoya/H.Lazaroi%20(H.293).json] {
  "error" : "Invalid path: Invalid token in path"
}


In [52]:
# for i in range(3):
name = "H . Lazaroi (H 293)"
data = {
    "Passport Data": {
        'Name': f"{name}.65"
    }
}
db.child('Hoya').child(f"{name}").set(data)

HTTPError: [Errno 400 Client Error: Bad Request for url: https://pnri-demeter-default-rtdb.firebaseio.com/Hoya/H%20.%20Lazaroi%20(H%20293).json] {
  "error" : "Invalid path: Invalid token in path"
}


In [58]:
for i in range(len(db_df)):
    name = db_df[0][i]

    pl = db_df[1][i]
    pw = db_df[2][i]
    rl = db_df[3][i]
    shl = db_df[4][i]
    wl = db_df[5][i]
    hl = db_df[6][i]
    el = db_df[7][i]
    tal = db_df[8][i]
    td = db_df[9][i]
    cb = db_df[10][i]

    scan_id = generate()

    qr_url = add_qr(name, scan_id)

    data = { 
        "Passport Data": {
            'Name': f'{name}',
            'Date Acquired':'',
            'Accession Origin': '',
            'Project': '',
            'Project Leader': '',
            'Other Details': '',
        },
        "Morphology":{
            'Pollinium Length': f'{pl}',
            "Pollinium Widest": f'{pw}',
            "Retinaculum Length": f'{rl}',
            "Shoulder": f'{shl}',
            "Waist": f'{wl}',
            "Hips": f'{hl}',
            "Extension": f'{el}',
            "Translator Arm Length": f'{tal}',
            "Translator Depth": f'{td}',
            'Caudicle Bulb Diameter': f'{cb}',
        },
        "urls":{
            'img_url' : '',
            'file_url' : '',
            'qr_url': f'{qr_url}',
        },
        'scan_id' : f'{scan_id}'
        }
    db.child('Hoya').child(f'{name}').set(data)


In [135]:
import pandas as pd

data = []
names = ["Asusss","HGYRKY","Hoya curtis","Talaga ba"]

for i in names:
    array = []
    morphology = (db.child("Hoya").child(i).child("Morphology").get())
    for datas in morphology.each():
        print(f"{datas.key()} : {datas.val()}")
        array.append(float(datas.val()))
    data.append(array)

sample = [
    [0.53,0.254,0.51,0.37],
    [1.04,0.4,0.62,0.43],
    [0.23,0.13,0.1,0.05],
    [0.39,0.15,0.15,0.08]
]
df = pd.DataFrame(sample)

matched_input = []
input = [0.5,0.25,0.52,0.39]

for i in range(len(df)):
    matched_input.append(input)

idf = pd.DataFrame(matched_input)


            # halign="center"


Caudicle Bulb Diameter : 545
Pollinium : 3535
Retinaculum : 3535
Translator : 3421
Caudicle Bulb Diameter : 3241
Pollinium : 134116
Retinaculum : 5352
Translator : 6209
Caudicle Bulb Diameter : 2427
Pollinium : 57952
Retinaculum : 9656
Translator : 2498
Caudicle Bulb Diameter : 125161
Pollinium : 13135
Retinaculum : 41325
Translator : 136146


In [136]:
df

,0,1,2,3
0,0.53,0.254,0.51,0.37
1,1.04,0.400,0.62,0.43
2,0.23,0.130,0.10,0.05
3,0.39,0.150,0.15,0.08


In [137]:
idf


,0,1,2,3
0,0.5,0.25,0.52,0.39
1,0.5,0.25,0.52,0.39
2,0.5,0.25,0.52,0.39
3,0.5,0.25,0.52,0.39


In [138]:
score = abs((idf-df)/((idf+df)/2))*100

In [139]:
score

,0,1,2,3
0,5.825243,1.587302,1.941748,5.263158
1,70.129870,46.153846,17.543860,9.756098
2,73.972603,63.157895,135.483871,154.545455
3,24.719101,50.000000,110.447761,131.914894


In [140]:
score['mean'] = score.mean(axis=1)
score

,0,1,2,3,mean
0,5.825243,1.587302,1.941748,5.263158,3.654362
1,70.129870,46.153846,17.543860,9.756098,35.895918
2,73.972603,63.157895,135.483871,154.545455,106.789956
3,24.719101,50.000000,110.447761,131.914894,79.270439


In [141]:
score['mean'].iloc[1:]

1     35.895918
2    106.789956
3     79.270439
Name: mean, dtype: float64

In [158]:
passed = score['mean'][score['mean'] <= 5].index.tolist()
passed

[0]

In [156]:
df[0:1].T

,0
0,0.530
1,0.254
2,0.510
3,0.370


In [166]:
for i in passed:
    # print(i)
    idf[i:i+1].T


In [165]:
idf[i:i+1].T

,0
0,0.50
1,0.25
2,0.52
3,0.39


In [157]:
from scipy.stats import ttest_ind
ttest_ind(df[0:1].T, idf[0:1].T)


Ttest_indResult(statistic=array([0.01116338]), pvalue=array([0.99145503]))

In [ ]:
from scipy.stats import ttest_ind

cat1 = my_data[my_data['Category']=='cat1']
cat2 = my_data[my_data['Category']=='cat2']

ttest_ind(cat1['values'], cat2['values'])
>>> (1.4927289925706944, 0.16970867501294376)